In [1]:
import pandas as pd
import json
from pandas import RangeIndex

In [2]:
csv_path = 'labels_my-project-name_2022-11-09-11-13-58.csv'
df = pd.read_csv(csv_path)

In [3]:
sort_ids = df['image_name'].value_counts().sort_index()
df['annotation_id'] = [x for i in sort_ids for x in RangeIndex(i)] 
df['image_id'] = df.groupby(['image_name']).ngroup()
df.head()

,label_name,bbox_x,bbox_y,bbox_width,bbox_height,image_name,image_width,image_height,annotation_id,image_id
0,riot_shield,695,44,289,540,000000.jpg,1280,720,0,0
1,baton,3,72,561,154,000000.jpg,1280,720,1,0
2,baton,255,0,336,308,000001.jpg,1280,720,0,1
3,riot_shield,695,46,288,534,000001.jpg,1280,720,1,1
4,riot_shield,697,44,287,538,000002.jpg,1280,720,0,2


In [4]:
categories = {
    'uniformed': 0,
    'non_uniformed': 1,
    'riot_shield': 2,
    'gun': 3,
    'pepper_spray': 4,
    'baton': 5,
    'chemical_smoke': 6
}

In [16]:
image_list = []
annotations_list = []

for index, row in df.iterrows():

    image_list.append({
        'id': row['image_id'],
        'width': row['width'],
        'height': row['height'],
        'file_name': row['image_name']
    })

    annotations_list.append({
        'id': row['annotation_id'],
        'image_id': row['image_id'],
        'category_id': categories.get(row['label_name']),
        'segmentation': [],
        'area': 0,
        'bbox': [
            row['bbox_x1'],
            row['bbox_y1'],
            row['bbox_x2'] - row['bbox_x1'],
            row['bbox_y2'] - row['bbox_y1']
        ],
        'iscrowd': 0   
    })

In [11]:
df.head(20)

,label_name,bbox_x,bbox_y,bbox_width,bbox_height,image_name,image_width,image_height,annotation_id,image_id
0,baton,1067,187,190,182,000000.jpg,1280,720,0,0
1,riot_shield,524,445,312,240,000000.jpg,1280,720,1,0
2,baton,698,283,46,56,000000.jpg,1280,720,2,0
3,baton,1038,175,241,195,000001.jpg,1280,720,0,1
4,riot_shield,523,431,323,238,000001.jpg,1280,720,1,1
5,baton,709,272,36,51,000001.jpg,1280,720,2,1
6,baton,1068,183,211,151,000002.jpg,1280,720,0,2
7,riot_shield,527,437,321,240,000002.jpg,1280,720,1,2
8,baton,715,255,30,53,000002.jpg,1280,720,2,2
9,baton,1093,220,154,102,000003.jpg,1280,720,0,3


In [5]:
predictions = []

for index, row in df.iterrows():
    predictions.append([
        row['image_id'],
        categories.get(row['label_name']),
        1,
        row['bbox_x'],
        row['bbox_y'],
        row['bbox_x'] + row['bbox_width'],
        row['bbox_y'] + row['bbox_height']
    ])
    

In [24]:
with open(f'ground_truth_3.json', 'w+') as f:
    json.dump(predictions, f)

In [17]:
json_store = {}

# build json in coco format
json_store['info'] = {
    'description': 'test'
}

json_store['images'] = image_list

json_store['annotations'] = annotations_list

json_store['categories'] = [
    {"id": 0, "name": "uniformed"},
    {"id": 1, "name": "non_uniformed"},
    {"id": 2, "name": "riot_shield"},
    {"id": 3, "name": "gun"},
    {"id": 4, "name": "pepper_spray"},
    {"id": 5, "name":" baton"},
    {"id": 6, "name": "chemical_smoke"}
]

In [18]:
# Write file, bbox and label data to json
with open(f'ground_truth.json', 'w+') as f:
    json.dump(json_store, f)